# 03_icoer_experiments.ipynb

**Experimentos com ICOER: Índice de Coerência Informacional**

Notebook para explorar a métrica ICOER e suas aplicações computacionais, conforme extensão da TGU para computação coerente (Seções 6.2 e 7.4 do artigo).

Objetivos:
- Calcular ICOER entre distribuições/estados
- Aplicar ativação coerente modulada por ICOER
- Treinar rede simples com regularização de coerência
- Observar impacto na eficiência informacional (menor entropia, maior estabilidade)

In [ ]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

# Importar módulos TGU
from src.icoer_torch import (
    ICOER,
    coherence_activation,
    icoer_regularization_loss,
    ResonanceGateLayer
)

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 1. Teste básico de ICOER entre dois estados

In [ ]:
# Estados dummy
batch_size, features = 64, 128
coherent_ref = torch.randn(batch_size, features) * 0.2 + 1.0  # referência "coerente"
noisy_state = coherent_ref + torch.randn_like(coherent_ref) * 1.5  # estado com ruído

icoer_calc = ICOER()

icoer_high = icoer_calc(coherent_ref, coherent_ref)
icoer_low  = icoer_calc(noisy_state, coherent_ref)

print(f"ICOER (estado coerente vs si mesmo): {icoer_high.item():.4f}")
print(f"ICOER (estado ruidoso vs referência): {icoer_low.item():.4f}")

## 2. Coherence Activation: amplificação seletiva

In [ ]:
# Ativação sem coerência (ReLU padrão)
x = torch.randn(batch_size, features) * 2.0
relu_out = F.relu(x)

# Com ICOER alto
icoer_high = icoer_calc(x, x)  # auto-coerência
coherent_out = coherence_activation(x, icoer_high, threshold=0.35)

print(f"Média ReLU padrão:          {relu_out.mean().item():.4f}")
print(f"Média com ativação coerente: {coherent_out.mean().item():.4f}")
print(f"Amplificação média:         {(coherent_out.mean() / relu_out.mean()):.3f}x")

## 3. Treinamento simples com regularização ICOER

In [ ]:
# Rede simples: autoencoder-like com ICOER
class SimpleCoherentNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 64)
        self.icoer = ICOER()
        self.res_gate = ResonanceGateLayer(32)

    def forward(self, x):
        z = F.relu(self.fc1(x))
        z_gated, icoer_mid = self.res_gate(z)
        out = self.fc2(z_gated)
        return out, icoer_mid

model = SimpleCoherentNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

# Dados dummy (reconstrução de sinal com ruído)
x_clean = torch.randn(256, 64).to(device)
x_noisy = x_clean + torch.randn_like(x_clean) * 0.8

losses, icoers = [], []

for epoch in range(200):
    optimizer.zero_grad()
    recon, icoer = model(x_noisy)
    
    recon_loss = F.mse_loss(recon, x_clean)
    coherence_reg = icoer_regularization_loss(icoer, target=0.85, weight=0.15)
    
    loss = recon_loss + coherence_reg
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    icoers.append(icoer.item())
    
    if epoch % 40 == 0:
        print(f"Epoch {epoch:3d} | Loss: {loss.item():.4f} | Recon: {recon_loss.item():.4f} | ICOER: {icoer.item():.4f}")

## 4. Visualização dos resultados do treinamento

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(losses, label='Total Loss', color='darkblue')
ax1.set_title('Evolução da Perda')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.grid(True, alpha=0.3)
ax1.legend()

ax2.plot(icoers, label='ICOER médio', color='darkgreen')
ax2.axhline(0.85, color='gray', linestyle='--', alpha=0.7, label='Target 0.85')
ax2.set_title('Evolução do ICOER durante treinamento')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('ICOER')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

## 5. Conclusões e próximos passos

- ICOER alto leva a ativações mais fortes e estáveis
- Regularização de coerência ajuda a manter estados alinhados, reduzindo entropia informacional
- Próximos experimentos:
  - Integrar com AYA-NODE (04_aya_node_prototype.ipynb)
  - Aplicar em tarefas reais (classificação, geração)
  - Medir eficiência energética (simulada) em treinamentos longos

ICOER como ferramenta de "strain informacional" em IA → caminho para computação mais coerente e sustentável.